In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("pevogam/ucf101")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/ucf101


In [ ]:
import shutil

# Move the extracted dataset to /content
shutil.copytree('/kaggle/input/ucf101', '/content/UCF101')


'/content/UCF101'

In [ ]:
import os
import cv2
import numpy as np
import random

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, GlobalAveragePooling3D, Dense


In [ ]:
# Settings
DATASET_PATH = '/content//UCF101/UCF101/UCF-101/'  # path where UCF101 extracted
selected_classes = ['Basketball', 'Typing', 'GolfSwing', 'WalkingWithDog', 'PlayingGuitar']
frames_per_video = 16
img_height, img_width = 112, 112

def load_ucf101_data(dataset_path, selected_classes, frames=16):
    X = []
    y = []

    for class_name in selected_classes:
        class_path = os.path.join(dataset_path, class_name)
        video_files = os.listdir(class_path)

        for video_file in video_files:
            video_path = os.path.join(class_path, video_file)
            frames_list = []

            cap = cv2.VideoCapture(video_path)
            total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

            if total_frames < frames:
                continue  # skip videos with too few frames

            frame_indices = np.linspace(0, total_frames-1, frames, dtype=int)

            for idx in frame_indices:
                cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
                ret, frame = cap.read()
                if ret:
                    frame = cv2.resize(frame, (img_width, img_height))
                    frame = frame / 255.0
                    frames_list.append(frame)
                else:
                    break

            cap.release()

            if len(frames_list) == frames:
                X.append(frames_list)
                y.append(class_name)

    return np.array(X), np.array(y)

# Load dataset
X, y = load_ucf101_data(DATASET_PATH, selected_classes, frames=frames_per_video)

print("Data loaded:", X.shape, y.shape)


Data loaded: (666, 16, 112, 112, 3) (666,)


In [ ]:
# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# One-hot encode
y_encoded = tf.keras.utils.to_categorical(y_encoded)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, stratify=y_encoded, random_state=42)

print("Train:", X_train.shape, y_train.shape)
print("Test:", X_test.shape, y_test.shape)

num_classes = len(selected_classes)


Train: (532, 16, 112, 112, 3) (532, 5)
Test: (134, 16, 112, 112, 3) (134, 5)


In [ ]:
# Define model
model = Sequential([
    Conv3D(32, (3,3,3), activation='relu', padding='same', input_shape=(frames_per_video, img_height, img_width, 3)),
    MaxPooling3D((2,2,2)),

    Conv3D(64, (3,3,3), activation='relu', padding='same'),
    MaxPooling3D((2,2,2)),

    Conv3D(128, (3,3,3), activation='relu', padding='same'),
    GlobalAveragePooling3D(),

    Dense(256, activation='relu'),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv3d (Conv3D)                 │ (None, 16, 112, 112,   │         2,624 │
│                                 │ 32)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d (MaxPooling3D)    │ (None, 8, 56, 56, 32)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_1 (Conv3D)               │ (None, 8, 56, 56, 64)  │        55,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_1 (MaxPooling3D)  │ (None, 4, 28, 28, 64)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_2 (Conv3D)               │ (None, 4, 28, 28, 128) │       221,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling3d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling3D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │         1,285 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 313,605 (1.20 MB)

 Trainable params: 313,605 (1.20 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.arange(num_classes), y=np.argmax(y_train, axis=1))
class_weights = dict(enumerate(class_weights))

print("Class weights:", class_weights)


Class weights: {0: np.float64(1.064), 1: np.float64(1.0431372549019609), 2: np.float64(0.83125), 3: np.float64(0.9761467889908257), 4: np.float64(1.1440860215053763)}


In [ ]:
# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=20,
    batch_size=8,
    class_weight=class_weights
)


Epoch 1/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 18s 162ms/step - accuracy: 0.1747 - loss: 1.6265 - val_accuracy: 0.3284 - val_loss: 1.5479
Epoch 2/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 5s 75ms/step - accuracy: 0.3163 - loss: 1.5046 - val_accuracy: 0.3284 - val_loss: 1.2487
Epoch 3/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 5s 75ms/step - accuracy: 0.4545 - loss: 1.2015 - val_accuracy: 0.4627 - val_loss: 1.1052
Epoch 4/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 5s 75ms/step - accuracy: 0.4773 - loss: 1.0975 - val_accuracy: 0.5299 - val_loss: 1.0906
Epoch 5/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 5s 76ms/step - accuracy: 0.5662 - loss: 1.0801 - val_accuracy: 0.5746 - val_loss: 0.9766
Epoch 6/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 5s 76ms/step - accuracy: 0.5508 - loss: 1.0174 - val_accuracy: 0.6343 - val_loss: 0.8725
Epoch 7/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 5s 77ms/step - accuracy: 0.6355 - loss: 0.9121 - val_accuracy: 0.6567 - val_loss: 0.8158
Epoch 8/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 10s 77ms/step - accuracy: 0.6552 - loss: 0.7995 - val_accuracy: 0.6791 

In [ ]:
# Evaluate on test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

# Predictions
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# Classification report
print("\nClassification Report:")
print(classification_report(y_true_classes, y_pred_classes, target_names=label_encoder.classes_))

# Confusion matrix
print("\nConfusion Matrix:")
print(confusion_matrix(y_true_classes, y_pred_classes))


5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 77ms/step - accuracy: 0.8118 - loss: 0.9082
Test Loss: 0.8519, Test Accuracy: 0.8209
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 152ms/step

Classification Report:
                precision    recall  f1-score   support

    Basketball       0.68      0.60      0.64        25
     GolfSwing       1.00      0.58      0.73        26
 PlayingGuitar       0.86      1.00      0.93        32
        Typing       0.93      1.00      0.96        27
WalkingWithDog       0.68      0.88      0.76        24

      accuracy                           0.82       134
     macro avg       0.83      0.81      0.81       134
  weighted avg       0.84      0.82      0.81       134


Confusion Matrix:
[[15  0  5  0  5]
 [ 4 15  0  2  5]
 [ 0  0 32  0  0]
 [ 0  0  0 27  0]
 [ 3  0  0  0 21]]
